In [81]:
import pandas as pd
import shutil
import os
import csv


def build_html_template(html_fields_path: str):
    """ Builds the html template based on the file h """


def get_heading(df: pd.DataFrame, cols=None):
    """ Accepts a pandas DataFrame and list of columns
        and creates a new 'heading' column.

        :df:
            pandas DataFrame
        :cols:
            list of columns used to make the 'heading'

        :returns: df with 'heading' column added
    """

    # change missing entries to empty strings
    df[cols] = df[cols].fillna('')

    df['heading'] = df[cols].agg('-'.join, axis=1)

    return df


def write_html(df: pd.DataFrame):
    """ Accepts a pandas DataFrame and writes an html file """
    
    with open('templates/template.html', 'r') as html_file:
        HTML = html_file.read()
        
    with open('html_fields.csv', 'r') as fields_file:
        reader = csv.reader(fields_file, delimiter=',')
        fields = next(reader)
        
    print(fields)
        
    # create column to store individual html markup
    df['html'] = HTML
    
    # fields to replace in the html template
#     fields = ['FirstName']


def make_discussions(path: str, program_name: str) -> int:
    """ Accepts a Microsoft Excel file path, builds pandas DataFrame with the 'heading' column,
        and creates the corresponding XML/HTML files for the D2L Discussions.
        
        Stores all HTML files in temp_folder until ready to zip.
    """
    
    try:
        os.mkdir('temp_folder')
    except FileExistsError:
        pass
    

    df = pd.read_excel(path)

    # the data from these columns will replace the corresponding fields in 'DiscussionTemplate.txt'
    cols = ['Person First Name', 'Person Last Name', 'Student ID', 'Acad Prog Code', 'Academic Plan 1 Code', 'ID', 'html_filename']
    fields = ['FirstName', 'LastName', 'StudentNumber', 'AcadProgram', 'AcademicPlan', 'IDnumberHERE', 'FILENAME']

    # if program_name == 'PHAS':
    #     # select the columns for PHAS program application discussions
    #     fields = ['Person First Name', 'Person Last Name', 'Student ID', 'Academic Plan 1 Code']
    #
    # else:
    #     # select the columns for MTST subprograms application discussions
    #     fields = ''
    
    # put names in proper case
    df['Person First Name'] = df['Person First Name'].str.title()
    df['Person Last Name'] = df['Person Last Name'].str.title()

    # convert student ID to string
    df['Student ID'] = df['Student ID'].apply(str)
    
#     df = get_heading(df, cols=cols)  
    
    # copy the index column to an ID column
    df['ID'] = df.index
    df['ID'] = df['ID'].apply(str)

    # create a new column of discussion topics for all students
    with open('templates/DiscussionTemplate.txt', 'r') as single_topic:
        text = single_topic.read()
    df['discussion_topic'] = text
    
    # create a filename column, replacing spaces with empty strings
    df['html_filename'] = df['Person First Name'].str.replace(' ', '') + df['Person Last Name'].str.replace(' ', '') + df['Student ID'] + '.html'

    # replace the fields in the discussion topic column with the corresponding student data
    for col, field in zip(cols, fields):
        df['discussion_topic'] = df.apply(lambda x: x['discussion_topic'].replace(field, x[col]), axis=1)
       
    make_forum(df[['discussion_topic', 'html_filename']], year_of_application=2022)  # TODO: allow year selection
    
    return df


def make_forum(df: pd.DataFrame, year_of_application: int):
    """ Accepts a pandas DataFrame of student discussions and creates the D2L 
        discussion forum object by filling in discussion_d2l_TemplateFile.txt
    """
    
    with open('templates/discussion_d2l_TemplateFile.txt', 'r') as all_students:
        forum = all_students.read()
        
    print(df)
        
    all_topics = ''.join(df['discussion_topic'].to_list())
    forum = forum.replace("InsertTopicHere", all_topics)
    forum = forum.replace("TOPIC_TITLE", str(year_of_application) + " Applications")
    
    with open('temp_folder/d2l_disc.xml', 'w') as file:
        file.write(forum)
        
    return 0

In [82]:
write_html(df)

['FirstName', 'LastName', 'StudentNumber', 'AcadProgram', 'AcademicPlan', 'Question1', 'Question2', 'RAparagraph', 'STparagraph', 'GPAparagraph', 'AWparagraph', 'FSparagraph', 'JEparagraph', 'PUBparagraph', 'OTparagraph', 'theGender', 'Bday', 'theCountry', 'theRes']


TypeError: 'NoneType' object does not support item assignment

In [83]:
df = make_discussions('PHAS Fall 2022 admission.xlsx', 'PHAS')

                                     discussion_topic  \
0   <topic id="0" resource_code="UCalgary-2643339"...   
1   <topic id="1" resource_code="UCalgary-2643339"...   
2   <topic id="2" resource_code="UCalgary-2643339"...   
3   <topic id="3" resource_code="UCalgary-2643339"...   
4   <topic id="4" resource_code="UCalgary-2643339"...   
5   <topic id="5" resource_code="UCalgary-2643339"...   
6   <topic id="6" resource_code="UCalgary-2643339"...   
7   <topic id="7" resource_code="UCalgary-2643339"...   
8   <topic id="8" resource_code="UCalgary-2643339"...   
9   <topic id="9" resource_code="UCalgary-2643339"...   
10  <topic id="10" resource_code="UCalgary-2643339...   
11  <topic id="11" resource_code="UCalgary-2643339...   
12  <topic id="12" resource_code="UCalgary-2643339...   
13  <topic id="13" resource_code="UCalgary-2643339...   
14  <topic id="14" resource_code="UCalgary-2643339...   
15  <topic id="15" resource_code="UCalgary-2643339...   
16  <topic id="16" resource_cod

In [84]:
make_forum(df, 2022)

   Acad Group Code Primary Acad Plan Code Acad Prog Code Degree Plan Code  \
0               GS               PHAS-AOS          GSDOC          PHD-DEG   
1               GS               PHAS-AOS          GSDOC          PHD-DEG   
2               GS               PHAS-AOS          GSDOC          PHD-DEG   
3               GS               PHAS-AOS          GSDOC          PHD-DEG   
4               GS               PHAS-AOS          GSDOC          PHD-DEG   
5               GS               PHAS-AOS          GSDOC          PHD-DEG   
6               GS               PHAS-AOS          GSMTH          MSC-DEG   
7               GS               PHAS-AOS          GSMTH          MSC-DEG   
8               GS               PHAS-AOS          GSMTH          MSC-DEG   
9               GS               PHAS-AOS          GSMTH          MSC-DEG   
10              GS               PHAS-AOS          GSMTH          MSC-DEG   
11              GS               PHAS-AOS          GSMTH          MSC-DEG   

0

In [85]:
df1 = pd.read_excel('PHAS Fall 2022 admission.xlsx')

In [87]:
df1.columns

Index(['Acad Group Code', 'Primary Acad Plan Code', 'Acad Prog Code',
       'Degree Plan Code', 'Academic Plan 1 Code', 'Academic Plan 2 Code',
       'Academic Plan 3 Code', 'Person First Name', 'Person Last Name',
       'Student ID', 'Mailing Address', 'Person Phone', 'Person Email Addr',
       'Gender', 'Person Disability Ind', 'Person Aboriginal Ind',
       'Birth Date', 'Country', 'Residency', 'Application Status Desc',
       'Ref 1 First Name', 'Ref 1 Last Name', 'Ref 1 Phone',
       'Ref 1 Organization Name', 'Ref 1 Title', 'Ref 2 First Name',
       'Ref 2 Last Name', 'Ref 2 Phone', 'Ref 2 Organization Name',
       'Ref 2 Title', 'Ref 3 First Name', 'Ref 3 Last Name', 'Ref 3 Phone',
       'Ref 3 Organization Name', 'Ref 3 Title',
       'Are you applying to a combined program? (Example: MBA/JD) If yes, specify which program. ',
       ' Are you looking for advanced credit? If yes, please list the courses you wish to receive credit for.  ',
       'Faculty Question 3', '